In [6]:
import pennylane as qml
import torch

N_MODES = 3
dev = qml.device("default.gaussian", wires=N_MODES)

@qml.qnode(dev, interface="torch")
def encode_voxels(x, disp_scale=1.0):
    """Encode voxel activations as CV states via Displacement + Squeezing."""
    for m in range(N_MODES):
        alpha = (x[m] - 0.5) * disp_scale
        qml.Displacement(alpha, 0.0, wires=m)   # encodes mean quadrature
        qml.Squeezing(0.0, 0.0, wires=m)        # no squeezing yet
    return [qml.expval(qml.X(m)) for m in range(N_MODES)]  # Changed QuadX to X

# test
x = torch.tensor([0.6, 0.4, 0.9])
print("Encodings (⟨x⟩):", encode_voxels(x))

Encodings (⟨x⟩): tensor([ 0.2000, -0.2000,  0.8000], dtype=torch.float64)


In [7]:
import pennylane as qml
import torch

N_MODES = 3
dev = qml.device("default.gaussian", wires=N_MODES)

@qml.qnode(dev, interface="torch")
def encode_with_phase(x, rot_params):
    for m in range(N_MODES):
        alpha = (x[m] - 0.5)
        qml.Displacement(alpha, 0.0, wires=m)
        qml.Rotation(rot_params[m], wires=m)   # Changed PhaseShift to Rotation for CV systems
    return [qml.expval(qml.X(m)) for m in range(N_MODES)]

# --- test
x = torch.tensor([0.6, 0.4, 0.9])
rot = torch.tensor([0.1, 0.2, -0.1])
print("With phase shifts:", encode_with_phase(x, rot))

With phase shifts: tensor([ 0.1990, -0.1960,  0.7960], dtype=torch.float64)


In [8]:
import pennylane as qml
import torch
import itertools

N_MODES = 3
dev = qml.device("default.gaussian", wires=N_MODES)

pairs = list(itertools.combinations(range(N_MODES), 2))

@qml.qnode(dev, interface="torch")
def cv_with_entanglement(x, rot_params, bs_params):
    # --- Encode + Phase rotation
    for m in range(N_MODES):
        qml.Displacement(x[m] - 0.5, 0.0, wires=m)
        qml.Rotation(rot_params[m], wires=m)

    # --- Entanglement via Beamsplitter
    # bs_params shape: (num_pairs, 2) -> (theta, phi) per pair
    for idx, (i, j) in enumerate(pairs):
        theta, phi = bs_params[idx]
        qml.Beamsplitter(theta, phi, wires=[i, j])

    # --- Simple readout: expectation of X quadrature
    return [qml.expval(qml.X(m)) for m in range(N_MODES)]

# --- test
x = torch.tensor([0.6, 0.4, 0.9])
rot = torch.tensor([0.1, 0.2, -0.1])
bs = torch.tensor([[0.2, 0.0], [0.3, 0.1], [0.1, -0.2]])  # 3 pairs for 3 modes

print("Entangled features:", cv_with_entanglement(x, rot, bs))


Entangled features: tensor([-0.0082, -0.2341,  0.8087], dtype=torch.float64)


In [8]:
# step4_qumodes_safe_loop.py
import pennylane as qml
import torch
import itertools

N_MODES = 3
dev = qml.device("default.gaussian", wires=N_MODES)

pairs = list(itertools.combinations(range(N_MODES), 2))

# Single-QNode that prepares the full circuit but returns only <X> for mode `m_idx`
def make_x_qnode(m_idx):
    @qml.qnode(dev, interface="torch")
    def x_qnode(x, rot_params, tms_params, bs_params=None):
        # encode + rotations
        for m in range(N_MODES):
            qml.Displacement(float(x[m]) - 0.5, 0.0, wires=m)
            qml.Rotation(float(rot_params[m]), wires=m)

        # Two-mode squeezing entanglers
        for idx, (i, j) in enumerate(pairs):
            if tms_params.ndim == 1:
                r = float(tms_params[idx])
                phi = 0.0
            else:
                r = float(tms_params[idx, 0])
                phi = float(tms_params[idx, 1])
            qml.TwoModeSqueezing(r, phi, wires=[i, j])

        # optional beamsplitter mixing
        if bs_params is not None:
            for idx, (i, j) in enumerate(pairs):
                theta = float(bs_params[idx, 0])
                phi_b  = float(bs_params[idx, 1])
                qml.Beamsplitter(theta, phi_b, wires=[i, j])

        return qml.expval(qml.X(m_idx))
    return x_qnode

# Single-QNode that prepares the full circuit but returns only <P> for mode `m_idx`
def make_p_qnode(m_idx):
    @qml.qnode(dev, interface="torch")
    def p_qnode(x, rot_params, tms_params, bs_params=None):
        # same preparation
        for m in range(N_MODES):
            qml.Displacement(float(x[m]) - 0.5, 0.0, wires=m)
            qml.Rotation(float(rot_params[m]), wires=m)

        for idx, (i, j) in enumerate(pairs):
            if tms_params.ndim == 1:
                r = float(tms_params[idx])
                phi = 0.0
            else:
                r = float(tms_params[idx, 0])
                phi = float(tms_params[idx, 1])
            qml.TwoModeSqueezing(r, phi, wires=[i, j])

        if bs_params is not None:
            for idx, (i, j) in enumerate(pairs):
                theta = float(bs_params[idx, 0])
                phi_b  = float(bs_params[idx, 1])
                qml.Beamsplitter(theta, phi_b, wires=[i, j])

        return qml.expval(qml.P(m_idx))
    return p_qnode

# pre-create QNodes for each mode (to avoid redeclaring on every call)
x_qnodes = [make_x_qnode(m) for m in range(N_MODES)]
p_qnodes = [make_p_qnode(m) for m in range(N_MODES)]


def cv_features_tms_heterodyne_safe(x, rot_params, tms_params, bs_params=None):
    """
    Safe wrapper that evaluates <X> and <P> per mode using separate QNodes.
    Returns list of length 2 * N_MODES: [<X0>, <P0>, <X1>, <P1>, ...]
    """
    # sanity checks
    assert len(rot_params) == N_MODES
    assert tms_params.shape[0] == len(pairs)

    feats = []
    # For each mode, call X-QNode then P-QNode (each QNode returns a scalar)
    for m in range(N_MODES):
        xval = x_qnodes[m](x, rot_params, tms_params, bs_params)
        pval = p_qnodes[m](x, rot_params, tms_params, bs_params)
        feats.append(xval)
        feats.append(pval)
    return feats


# --- quick test
if __name__ == "__main__":
    x = torch.tensor([0.6, 0.4, 0.9])
    rot = torch.tensor([0.05, -0.02, 0.1])
    tms = torch.tensor([0.2, 0.15, 0.12])  # 3 pairs for N=3
    bs  = torch.tensor([[0.05, 0.0], [0.03, 0.0], [0.02, 0.0]])

    out = cv_features_tms_heterodyne_safe(x, rot, tms, bs_params=bs)
    print("Features (X0,P0,X1,P1,...):", out)


Features (X0,P0,X1,P1,...): [tensor(0.2635, dtype=torch.float64), tensor(-0.0045, dtype=torch.float64), tensor(-0.0673, dtype=torch.float64), tensor(-0.0092, dtype=torch.float64), tensor(0.8230, dtype=torch.float64), tensor(0.0794, dtype=torch.float64)]


In [5]:
import pennylane as qml
import torch
import itertools

N_MODES = 3
dev = qml.device("default.gaussian", wires=N_MODES)

pairs = list(itertools.combinations(range(N_MODES), 2))

def make_x_qnode(m_idx):
    @qml.qnode(dev, interface="torch")
    def x_qnode(x, rot_params, tms_params, bs_params=None):
        # encode + rotations
        for m in range(N_MODES):
            qml.Displacement(float(x[m]) - 0.5, 0.0, wires=m)
            qml.Rotation(float(rot_params[m]), wires=m)

        # Two-mode squeezing entanglers
        for idx, (i, j) in enumerate(pairs):
            if tms_params.ndim == 1:
                r = float(tms_params[idx])
                phi = 0.0
            else:
                r = float(tms_params[idx, 0])
                phi = float(tms_params[idx, 1])
            qml.TwoModeSqueezing(r, phi, wires=[i, j])

        # optional beamsplitter mixing
        if bs_params is not None:
            for idx, (i, j) in enumerate(pairs):
                theta = float(bs_params[idx, 0])
                phi_b  = float(bs_params[idx, 1])
                qml.Beamsplitter(theta, phi_b, wires=[i, j])

        return qml.expval(qml.X(m_idx))
    return x_qnode

def make_p_qnode(m_idx):
    @qml.qnode(dev, interface="torch")
    def p_qnode(x, rot_params, tms_params, bs_params=None):
        # same preparation
        for m in range(N_MODES):
            qml.Displacement(float(x[m]) - 0.5, 0.0, wires=m)
            qml.Rotation(float(rot_params[m]), wires=m)

        for idx, (i, j) in enumerate(pairs):
            if tms_params.ndim == 1:
                r = float(tms_params[idx])
                phi = 0.0
            else:
                r = float(tms_params[idx, 0])
                phi = float(tms_params[idx, 1])
            qml.TwoModeSqueezing(r, phi, wires=[i, j])

        if bs_params is not None:
            for idx, (i, j) in enumerate(pairs):
                theta = float(bs_params[idx, 0])
                phi_b  = float(bs_params[idx, 1])
                qml.Beamsplitter(theta, phi_b, wires=[i, j])

        return qml.expval(qml.P(m_idx))
    return p_qnode

# pre-create QNodes for each mode (to avoid redeclaring on every call)
x_qnodes = [make_x_qnode(m) for m in range(N_MODES)]
p_qnodes = [make_p_qnode(m) for m in range(N_MODES)]


def cv_features_tms_heterodyne_safe(x, rot_params, tms_params, bs_params=None):
    """
    Safe wrapper that evaluates <X> and <P> per mode using separate QNodes.
    Returns list of length 2 * N_MODES: [<X0>, <P0>, <X1>, <P1>, ...]
    """
    # sanity checks
    assert len(rot_params) == N_MODES
    assert tms_params.shape[0] == len(pairs)

    feats = []
    # For each mode, call X-QNode then P-QNode (each QNode returns a scalar)
    for m in range(N_MODES):
        xval = x_qnodes[m](x, rot_params, tms_params, bs_params)
        pval = p_qnodes[m](x, rot_params, tms_params, bs_params)
        feats.append(xval)
        feats.append(pval)
    return feats


# --- quick test
if __name__ == "__main__":
    x = torch.tensor([0.6, 0.4, 0.9])
    rot = torch.tensor([0.05, -0.02, 0.1])
    tms = torch.tensor([0.2, 0.15, 0.12])  # 3 pairs for N=3
    bs  = torch.tensor([[0.05, 0.0], [0.03, 0.0], [0.02, 0.0]])

    out = cv_features_tms_heterodyne_safe(x, rot, tms, bs_params=bs)
    print("Features (X0,P0,X1,P1,...):", out)


Features (X0,P0,X1,P1,...): [tensor(0.2635, dtype=torch.float64), tensor(-0.0045, dtype=torch.float64), tensor(-0.0673, dtype=torch.float64), tensor(-0.0092, dtype=torch.float64), tensor(0.8230, dtype=torch.float64), tensor(0.0794, dtype=torch.float64)]


In [4]:
# step5_qumodes_train.py
import torch
import torch.nn as nn
import torch.optim as optim

N_MODES = 3  # Assuming N_MODES is 3 based on the code context

def cv_features_tms_heterodyne_safe(xi, rot, tms, bs=None):
    # This is a simplified mock function that returns some values
    # Replace this with actual implementation if available
    return torch.randn(2 * N_MODES)  # Returns random features of expected size

# -----------------------
# Wrapper: extract CV features (no gradients through QNode)
# -----------------------
def extract_features(X, rot, tms, bs=None):
    feats = []
    for xi in X:
        f = cv_features_tms_heterodyne_safe(xi, rot, tms, bs)
        feats.append(torch.tensor(f, dtype=torch.float32))
    return torch.stack(feats)  # shape (batch, 2*N_MODES)

# -----------------------
# Simple classical regressor
# -----------------------
class ClassicalRegressor(nn.Module):
    def __init__(self, in_dim, out_dim=1):
        super().__init__()
        self.fc = nn.Linear(in_dim, out_dim)

    def forward(self, x):
        return self.fc(x)

# -----------------------
# Training demo
# -----------------------
if __name__ == "__main__":
    # params for the quantum feature map (fixed)
    rot = torch.tensor([0.05, -0.02, 0.1])
    tms = torch.tensor([0.2, 0.15, 0.12])
    bs  = torch.tensor([[0.05, 0.0], [0.03, 0.0], [0.02, 0.0]])

    # dataset
    X = torch.rand((40, N_MODES))
    y = X.sum(dim=1, keepdim=True)

    # feature extraction
    feats = extract_features(X, rot, tms, bs)

    # regressor
    model = ClassicalRegressor(in_dim=2 * N_MODES)
    opt = optim.Adam(model.parameters(), lr=0.05)
    loss_fn = nn.MSELoss()

    for epoch in range(1, 6):
        opt.zero_grad()
        pred = model(feats)
        loss = loss_fn(pred, y)
        loss.backward()
        opt.step()
        print(f"[QUMODES] Epoch {epoch}, Loss = {loss.item():.4f}")

[QUMODES] Epoch 1, Loss = 2.2721
[QUMODES] Epoch 2, Loss = 1.9137
[QUMODES] Epoch 3, Loss = 1.6090
[QUMODES] Epoch 4, Loss = 1.3532
[QUMODES] Epoch 5, Loss = 1.1433


/tmp/ipykernel_141/4144923229.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  feats.append(torch.tensor(f, dtype=torch.float32))


In [3]:
if __name__ == "__main__":
    rot = torch.tensor([0.05, -0.02, 0.1])
    tms = torch.tensor([0.2, 0.15, 0.12])
    bs  = torch.tensor([[0.05, 0.0], [0.03, 0.0], [0.02, 0.0]])

    X = torch.rand((40, N_MODES))
    y = X.sum(dim=1, keepdim=True)

    feats = extract_features(X, rot, tms, bs)

    model = ClassicalRegressor(in_dim=2 * N_MODES)
    opt = optim.Adam(model.parameters(), lr=0.05)
    loss_fn = nn.MSELoss()

    for epoch in range(1, 6):
        opt.zero_grad()
        pred = model(feats)
        loss = loss_fn(pred, y)
        loss.backward()
        opt.step()
        print(f"[QUMODES] Epoch {epoch}, Loss = {loss.item():.4f}")


[QUMODES] Epoch 1, Loss = 2.8471
[QUMODES] Epoch 2, Loss = 2.6159
[QUMODES] Epoch 3, Loss = 2.4094
[QUMODES] Epoch 4, Loss = 2.2339
[QUMODES] Epoch 5, Loss = 2.0833


/tmp/ipykernel_141/3680003830.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  feats.append(torch.tensor(f, dtype=torch.float32))
